In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

In [2]:
import nltk

from nltk.tokenize import word_tokenize
import string

from string import digits
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [3]:
df = pd.read_csv("final_data2.csv")

In [4]:
df.drop("Unnamed: 0", inplace=True, axis=1)

## EDA

In [5]:
df.shape

(15454, 8)

In [6]:
df.head()

,date,date_2,neighb,rating,rating_num,rev,title,city
0,"October 9, 2018",2018-10-09,chelsea,"['ui_bubble_rating', 'bubble_40']",4.0,"This is al ovely area with lots of cool shops,...",Lovely Area,nyc
1,"October 2, 2018",2018-10-02,chelsea,"['ui_bubble_rating', 'bubble_50']",5.0,my hotel was there/ it is very close to everyt...,wonderful!!,nyc
2,"August 11, 2018",2018-08-11,chelsea,"['ui_bubble_rating', 'bubble_40']",4.0,Getting down from High Line and you hit Chelse...,A special neighborhood,nyc
3,"May 27, 2018",2018-05-27,chelsea,"['ui_bubble_rating', 'bubble_40']",4.0,Stopped here on the afternoon. Beautiful resid...,Chelsea,nyc
4,"May 13, 2018",2018-05-13,chelsea,"['ui_bubble_rating', 'bubble_40']",4.0,This is very good neighbourhood here.Lots of f...,Flower,nyc


In [7]:
df["rev_len"] = df["rev"].apply(lambda x: len(x.split(" ")))

In [8]:
df["rev_len"].mean()

55.756114921703116

In [9]:
df[df["city"]=="nyc"].groupby("neighb")["title"].count()

neighb
chelsea               211
chinatown             971
east_village          150
fidi                  579
fifth_ave             500
greenwich_village    1068
harlem                306
soho                  998
times_sq              992
tribeca               121
ues                   111
union_sq              724
uws                   398
west_village          827
williamsburg          770
Name: title, dtype: int64

In [10]:
df[df["city"]=="SF"].groupby("neighb")["title"].count()

neighb
castro               426
chinatown_sf         999
embarcadero          619
fisherman_wharf      999
haight_ashbury       930
mission_district     325
nob_hill             137
north_becah          212
pacific_heights       67
presidio            1001
soma                  34
union_sq_sf          979
Name: title, dtype: int64

In [11]:
df[df["city"]=="nyc"].groupby("neighb")["rating_num"].mean().sort_values(ascending=False)

neighb
west_village         4.675937
greenwich_village    4.607678
fidi                 4.554404
uws                  4.550251
soho                 4.509018
fifth_ave            4.486000
chelsea              4.417062
williamsburg         4.411688
east_village         4.406667
tribeca              4.305785
ues                  4.297297
union_sq             4.285912
times_sq             4.237903
harlem               4.166667
chinatown            3.606591
Name: rating_num, dtype: float64

In [12]:
df[df["city"]=="SF"].groupby("neighb")["rating_num"].mean().sort_values(ascending=False)

neighb
presidio            4.590410
pacific_heights     4.388060
nob_hill            4.357664
castro              4.356808
embarcadero         4.306947
north_becah         4.301887
mission_district    4.030769
union_sq_sf         3.991828
fisherman_wharf     3.973974
soma                3.970588
chinatown_sf        3.944945
haight_ashbury      3.918280
Name: rating_num, dtype: float64

In [13]:
df.groupby("rating_num")["title"].count()

rating_num
1.0     270
2.0     491
3.0    1870
4.0    5098
5.0    7725
Name: title, dtype: int64

## Topic Modeling

In [14]:
df = df[df["neighb"]!="market_st"]

In [15]:
df.shape

(15454, 9)

In [16]:
#collect additional stop words
#     NYC Landmarks:
nyc_landmarks = ["'",'Williamsburg','arch','avenu','batteri','bedford','brooklyn','central','chelsea','china','chinatown','chines','christma','citi',
 'columbus','east','ferri','greenwich','harlem','island','lincoln','lower','manhattan','new','ny','nyc','nyu','riverside',
 'soho','tri','union','upper','villag','village','wall','washington','west','williamsburg','york', 'tour', 'bus', 'guid', 'hop', 'free', 'book']              
    
    #     San Francisco Landmarks
sf_landmarks = ['alcatraz','along','base','bay','bridg','cabl','car','castro','embarcadero','fisherman','francisco','gate',
 'golden','haightashburi','high','hill','militari','military','north','old','pier',"haight",'presidio','san','sf','wharf']

    #     Generic Travel Words
generic_travel = ['area','atmospher','ave','bit','block','build','buy','cafe','center','close','coffe','coffee',
 'crowd','culture','culture','day','district','drink','eat','experi','explor','far','food','found','fruit','full','go',
 'home','homeless','hotel','hour','hous','line','live','local','locat','long','look','lot','love','lunch','market','need',
 'neighborhood','neighbourhood','night','peopl','photo','place','plenti','price','restaur','restaur shop','safe',"music", "coupl", "life", "two", "famous",
 'see','sell','shop','shop restaur','side','sit','south','souvenir','spend','spent','spot','squar','cheap', 'sever', 'start', 'itali', 'vendor', 'especi',
 'st','stay','step','street','stroll','subway','tourist',"beach", "sea", "disney", 'town','trip','unique','use','walk','wander','watch','went']
    
    
    #     Generic Descriptive Words
descrip_words = ['also',"though",'also','pretti','alway','amaz','show', 'chang','amaz','around','away','awesom','back','best','best','better','big','charact','check',
 'check','come','could','definit','defint','didnt','differ','dont','easi','els','end','enjoy','etc','even',
     'everi','everyth','expect','feel','felt','find','first','friend','fun','get','good','got','great',
     'interest','know','light','liketime','littl','major','make','mani','miss','much','must','never',
     'nice','noth','number','offer','one','open','part','produc','quit','real','realli','recommend','right',
     'saturday','say','see','seem','sight','small','someth','still','stop','sunday','sure','take','time', 'next',"attract", "favorit", "made", "everyon",
 'th','thing','think','took','us','visit','walk','want','way','well','wonder','world','worth','would','year','’','like', 'anoth', 'kind', 'movi', 'youll']
              

In [17]:
stop_words = stopwords.words('english')

for word in nyc_landmarks:
    stop_words.append(word)
    
for word in sf_landmarks:
    stop_words.append(word)

for word in generic_travel:
    stop_words.append(word)
    
for word in descrip_words:
    stop_words.append(word)

In [18]:
def custom_tokenizer(text):

    # remove punctuation
    remove_punct = str.maketrans('', '', string.punctuation)
    text = text.translate(remove_punct)

    # remove digits and convert to lower case
    remove_digits = str.maketrans('', '', string.digits)
    text = text.lower().translate(remove_digits)

    # tokenize
    tokens = word_tokenize(text)

    # remove stop words
    tokens_stop = [y for y in tokens if y not in stop_words]

    # stem
    stemmer = SnowballStemmer('english')
    tokens_stem = [stemmer.stem(y) for y in tokens_stop] 

    return tokens_stem

In [19]:
from sklearn.feature_extraction.text import  TfidfVectorizer 

tfidf = TfidfVectorizer(tokenizer = custom_tokenizer,stop_words = stop_words, token_pattern = "\\b[a-zA-Z][a-zA-Z]+\\b", min_df = .01, max_df = .8 )
tfidf_vecs = tfidf.fit_transform(df["rev"])


/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'doesnt', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldv', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youv'] not in stop_words.
  sorted(inconsistent))


In [20]:
tfidf_2 = tfidf_vecs.todense()

In [21]:
tfidf_df2 = pd.DataFrame(tfidf_vecs.todense(), columns = tfidf.get_feature_names())

In [22]:
tfidf_df2.shape

(15454, 250)

In [23]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=14)
nmf_vecs = nmf.fit_transform(tfidf_vecs)

In [24]:
nmf_vecs.shape

(15454, 14)

In [25]:
topic_words = []
for r in nmf.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:6]
    topic_words.append([tfidf_df2.iloc[:,a[e][1]].name for e in range(len(a))])

In [26]:
for i in topic_words:
    print(i)

['park', 'play', 'dog', 'near', 'memori', 'run']
['store', 'cloth', 'maci', 'window', 'vintag', 'fifth']
['view', 'water', 'bike', 'top', 'across', 'point']
['bar', 'club', 'gay', 'trendi', 'eateri', 'hang']
['beauti', 'tree', 'brownston', 'absolut', 'clean', 'pictur']
['busi', 'bustl', 'keep', 'entertain', 'expens', 'activ']
['histori', 'histor', 'natur', 'gay', 'american', 'hippi']
['cool', 'trendi', 'stuff', 'hip', 'hang', 'vintag']
['dirti', 'everywher', 'howev', 'seen', 'saw', 'expens']
['architectur', 'histor', 'charm', 'design', 'style', 'lost']
['museum', 'famili', 'histor', 'includ', 'natur', 'memori']
['art', 'galleri', 'boutiqu', 'mission', 'mural', 'artist']
['vibe', 'hippi', 'young', 'trendi', 'boutiqu', 'eateri']
['relax', 'afternoon', 'surround', 'perfect', 'bustl', 'nearbi']


### Categories
0.park
1.store, shopping
2.views
3.bars/ going out
4.beautiful/ picturesque
5.busy, bustling
6.historical
7.cool, trendy 13. vibe, young
8.RECATEGORIZE
9.architecture
10.museum
11.art galleries
12.vibe, young
13.relax

## Assign Reviews to Clusters

In [27]:
doc_cluster = nmf_vecs.argmax(axis = 1)

In [28]:
print(doc_cluster.shape)
print(df.shape)

(15454,)
(15454, 9)


In [29]:
clusters1 = pd.DataFrame(doc_cluster)

In [30]:
print(clusters1.shape)

clusters1.reset_index(inplace=True)

(15454, 1)


In [31]:
# df.drop("index", axis=1, inplace=True)

In [32]:
df.reset_index(inplace=True)

In [33]:
df2 = df.merge(clusters1, how="outer", on="index")

In [34]:
# df2 = df2[df2[0]!=8]

In [35]:
df3 = df2.groupby(["neighb","city", 0])["index"].count().reset_index()

In [36]:
df3.neighb.unique()

array(['castro', 'chelsea', 'chinatown', 'chinatown_sf', 'east_village',
       'embarcadero', 'fidi', 'fifth_ave', 'fisherman_wharf',
       'greenwich_village', 'haight_ashbury', 'harlem',
       'mission_district', 'nob_hill', 'north_becah', 'pacific_heights',
       'presidio', 'soho', 'soma', 'times_sq', 'tribeca', 'ues',
       'union_sq', 'union_sq_sf', 'uws', 'west_village', 'williamsburg'],
      dtype=object)

In [37]:
df3.groupby("neighb")["index"].sum()

neighb
castro                426
chelsea               211
chinatown             971
chinatown_sf          999
east_village          150
embarcadero           619
fidi                  579
fifth_ave             500
fisherman_wharf       999
greenwich_village    1068
haight_ashbury        930
harlem                306
mission_district      325
nob_hill              137
north_becah           212
pacific_heights        67
presidio             1001
soho                  998
soma                   34
times_sq              992
tribeca               121
ues                   111
union_sq              724
union_sq_sf           979
uws                   398
west_village          827
williamsburg          770
Name: index, dtype: int64

In [38]:
df3.loc[df3["neighb"]=="castro", "total"]=426
df3.loc[df3["neighb"]=="chelsea", "total"]=211
df3.loc[df3["neighb"]=="chinatown", "total"]=971
df3.loc[df3["neighb"]=="chinatown_sf", "total"]=999
df3.loc[df3["neighb"]=="east_village", "total"]=150
df3.loc[df3["neighb"]=="embarcadero", "total"]=619
df3.loc[df3["neighb"]=="fidi", "total"]=579
df3.loc[df3["neighb"]=="fifth_ave", "total"]=500
df3.loc[df3["neighb"]=="fisherman_wharf", "total"]=999
df3.loc[df3["neighb"]=="greenwich_village", "total"]=1068
df3.loc[df3["neighb"]=="haight_ashbury", "total"]=930
df3.loc[df3["neighb"]=="harlem", "total"]=306
df3.loc[df3["neighb"]=="mission_district", "total"]=325
df3.loc[df3["neighb"]=="nob_hill", "total"]=137
df3.loc[df3["neighb"]=="north_becah", "total"]=212
df3.loc[df3["neighb"]=="pacific_heights", "total"]=67
df3.loc[df3["neighb"]=="presidio", "total"]=1001
df3.loc[df3["neighb"]=="soho", "total"]=998
df3.loc[df3["neighb"]=="soma", "total"]=34
df3.loc[df3["neighb"]=="times_sq", "total"]=992
df3.loc[df3["neighb"]=="tribeca", "total"]=121
df3.loc[df3["neighb"]=="ues", "total"]=111
df3.loc[df3["neighb"]=="union_sq", "total"]=724
df3.loc[df3["neighb"]=="union_sq_sf", "total"]=979
df3.loc[df3["neighb"]=="uws", "total"]=398
df3.loc[df3["neighb"]=="west_village", "total"]=827
df3.loc[df3["neighb"]=="williamsburg", "total"]=770

In [39]:
df3["perc"] = df3["index"] / df3["total"]

In [40]:
df3.neighb.nunique()

27

In [41]:
df3.loc[df3[0]==0,"cat"] = "park"
df3.loc[df3[0]==1,"cat"] = "shopping"
df3.loc[df3[0]==2,"cat"] = "views"
df3.loc[df3[0]==3,"cat"] = "bars"
df3.loc[df3[0]==4,"cat"] = "beautiful"
df3.loc[df3[0]==5,"cat"] = "busy/ bustling"
df3.loc[df3[0]==6,"cat"] = "historical"
df3.loc[df3[0]==7,"cat"] = "cool"
df3.loc[df3[0]==8,"cat"] = "???"
df3.loc[df3[0]==9,"cat"] = "architecture"
df3.loc[df3[0]==10,"cat"] = "museums"
df3.loc[df3[0]==11,"cat"] = "art"
df3.loc[df3[0]==12,"cat"] = "young vibe"
df3.loc[df3[0]==13,"cat"] = "relaxing"

In [42]:
df3.neighb.unique()

array(['castro', 'chelsea', 'chinatown', 'chinatown_sf', 'east_village',
       'embarcadero', 'fidi', 'fifth_ave', 'fisherman_wharf',
       'greenwich_village', 'haight_ashbury', 'harlem',
       'mission_district', 'nob_hill', 'north_becah', 'pacific_heights',
       'presidio', 'soho', 'soma', 'times_sq', 'tribeca', 'ues',
       'union_sq', 'union_sq_sf', 'uws', 'west_village', 'williamsburg'],
      dtype=object)

In [43]:
df3.cat.unique()

array(['park', 'shopping', 'views', 'bars', 'beautiful', 'busy/ bustling',
       'historical', 'cool', '???', 'architecture', 'museums', 'art',
       'young vibe', 'relaxing'], dtype=object)

In [44]:
df3.groupby("neighb")["perc"].sum()

neighb
castro               1.0
chelsea              1.0
chinatown            1.0
chinatown_sf         1.0
east_village         1.0
embarcadero          1.0
fidi                 1.0
fifth_ave            1.0
fisherman_wharf      1.0
greenwich_village    1.0
haight_ashbury       1.0
harlem               1.0
mission_district     1.0
nob_hill             1.0
north_becah          1.0
pacific_heights      1.0
presidio             1.0
soho                 1.0
soma                 1.0
times_sq             1.0
tribeca              1.0
ues                  1.0
union_sq             1.0
union_sq_sf          1.0
uws                  1.0
west_village         1.0
williamsburg         1.0
Name: perc, dtype: float64

In [49]:
def get_rec_by_attr(attr1, attr2, city):
    ser1 = df3[(df3["cat"]==attr1) & (df3["city"]==city)][["neighb", "perc"]]
    ser2 = df3[(df3["cat"]==attr2) & (df3["city"]==city)][["neighb", "perc"]]
    ser_comb = ser1.merge(ser2, how="outer", on="neighb")
    ser_comb["perc_z"] = (ser_comb["perc_x"] + ser_comb["perc_y"])/2
    ser_comb.sort_values(by="perc_z", ascending=False, inplace=True)
    return ser_comb.iloc[0:2]["neighb"].values

In [49]:
df3.to_csv("cat_perc.csv")

In [50]:
df3 = df3[df3[0]!=8]

## Try Topic Modeling w/ full reviews as corpus

In [51]:
neighbs = df.neighb.unique()

In [52]:
len(neighbs)

27

In [53]:
df.neighb.unique()

array(['chelsea', 'chinatown', 'east_village', 'fidi', 'fifth_ave',
       'greenwich_village', 'harlem', 'soho', 'times_sq', 'tribeca',
       'union_sq', 'ues', 'uws', 'west_village', 'williamsburg', 'castro',
       'chinatown_sf', 'embarcadero', 'fisherman_wharf', 'haight_ashbury',
       'mission_district', 'nob_hill', 'north_becah', 'pacific_heights',
       'presidio', 'soma', 'union_sq_sf'], dtype=object)

In [54]:
neighb_names = []

In [55]:
for i in neighbs:
    neighb_names.append(i+"_revs")

In [56]:
chelsea_revs = []
chinatown_revs = []
east_village_revs = []
fidi_revs = []
fifth_ave_revs = []
greenwich_village_revs = []
harlem_revs = []
soho_revs = []
times_sq_revs = []
tribeca_revs = []
union_sq_revs = []
ues_revs = []
uws_revs = []
west_village_revs = []
williamsburg_revs = []
castro_revs = []
chinatown_sf_revs = []
embarcadero_revs = []
fisherman_wharf_revs = []
haight_ashbury_revs = []
mission_district_revs = []
nob_hill_revs = []
north_becah_revs = []
pacific_heights_revs = []
presidio_revs = []
soma_revs = []
union_sq_sf_revs = []

In [57]:
def get_revs(x, neigh):
    length = df[df["neighb"]==neigh].shape[0]
    for i in range(0,length):
        x.append(df[df["neighb"]==neigh].iloc[i].rev)
        

In [58]:
get_revs(chelsea_revs, "chelsea")

In [59]:
len(chelsea_revs)

211

In [60]:
get_revs(east_village_revs, "east_village")

In [61]:
get_revs(chinatown_revs, "chinatown")
get_revs(east_village_revs, "east_village")
get_revs(fidi_revs, "fidi")
get_revs(fifth_ave_revs, "fifth_ave")
get_revs(greenwich_village_revs, "greenwich_village")
get_revs(harlem_revs, "harlem")
get_revs(soho_revs, "soho")
get_revs(times_sq_revs, "times_sq")
get_revs(tribeca_revs, "tribeca")
get_revs(union_sq_revs, "union_sq")
get_revs(ues_revs, "ues")
get_revs(uws_revs, "uws")
get_revs(west_village_revs, "west_village")
get_revs(williamsburg_revs, "williamsburg")
get_revs(castro_revs, "castro")
get_revs(chinatown_sf_revs, "chinatown_sf")
get_revs(embarcadero_revs, "embarcadero")
get_revs(fisherman_wharf_revs, "fisherman_wharf")
get_revs(haight_ashbury_revs, "haight_ashbury")
get_revs(mission_district_revs, "mission_district")
get_revs(nob_hill_revs, "nob_hill")
get_revs(north_becah_revs, "north_becah")
get_revs(pacific_heights_revs, "pacific_heights")
get_revs(presidio_revs, "presidio")
get_revs(soma_revs, "soma")
get_revs(union_sq_sf_revs, "union_sq_sf")

In [62]:
neighbs

array(['chelsea', 'chinatown', 'east_village', 'fidi', 'fifth_ave',
       'greenwich_village', 'harlem', 'soho', 'times_sq', 'tribeca',
       'union_sq', 'ues', 'uws', 'west_village', 'williamsburg', 'castro',
       'chinatown_sf', 'embarcadero', 'fisherman_wharf', 'haight_ashbury',
       'mission_district', 'nob_hill', 'north_becah', 'pacific_heights',
       'presidio', 'soma', 'union_sq_sf'], dtype=object)

In [63]:
chelsea_revs2 = []

In [64]:
chelsea_revs2 = " ".join(chelsea_revs)


In [65]:
chinatown_revs2 = " ".join(chinatown_revs)
east_village_revs2 = " ".join(east_village_revs)
fidi_revs2 = "".join(fidi_revs)
fifth_ave_revs2 = "".join(fifth_ave_revs)
greenwich_village_revs2 = " ".join(greenwich_village_revs)
harlem_revs2 = " ".join(harlem_revs)
soho_revs2 = " ".join(soho_revs)
times_sq_revs2 = " ".join(times_sq_revs)
tribeca_revs2 = " ".join(tribeca_revs)
union_sq_revs2 = " ".join(union_sq_revs)
ues_revs2 = " ".join(ues_revs)
uws_revs2 = " ".join(uws_revs)
west_village_revs2 = " ".join(west_village_revs)
williamsburg_revs2 = " ".join(williamsburg_revs)
castro_revs2 = " ".join(castro_revs)
chinatown_sf_revs2 = " ".join(chinatown_sf_revs)
embarcadero_revs2 = " ".join(embarcadero_revs)
fisherman_wharf_revs2 = " ".join(fisherman_wharf_revs)
haight_ashbury_revs2 = " ".join(haight_ashbury_revs)
mission_district_revs2 = " ".join(mission_district_revs)
nob_hill_revs2 = " ".join(nob_hill_revs)
north_becah_revs2 = " ".join(north_becah_revs)
pacific_heights_revs2 = " ".join(pacific_heights_revs)
presidio_revs2 = " ".join(presidio_revs)
soma_revs2 = " ".join(soma_revs)
union_sq_sf_revs2 = " ".join(union_sq_sf_revs)

In [66]:
df4 = pd.DataFrame({"neighb":["chelsea","chinatown","east_village","fidi","fifth_ave","greenwich_village","harlem","soho","times_sq","tribeca","union_sq","ues","uws","west_village","williamsburg","castro","chinatown_sf","embarcadero","fisherman_wharf","haight_ashbury","mission_district","nob_hill","north_beach","pacific_heights","presidio","soma","union_sq_sf"],
                    "revs":[chelsea_revs2,chinatown_revs2,east_village_revs2,fidi_revs2,fifth_ave_revs2,greenwich_village_revs2,harlem_revs2,soho_revs2,times_sq_revs2,tribeca_revs2,union_sq_revs2,ues_revs2,uws_revs2, west_village_revs2,williamsburg_revs2,castro_revs2,chinatown_sf_revs2,embarcadero_revs2,fisherman_wharf_revs2,haight_ashbury_revs2,mission_district_revs2,nob_hill_revs2,north_becah_revs2,pacific_heights_revs2,presidio_revs2,soma_revs2,union_sq_sf_revs2]})



In [67]:
add_words = ["bleeker", "fifth", "pacif", "height", "doubtfir", "mrs", "soma", "yerba", "buena", "tribeca", "nob",'bleecker', "mission", "ue", 'madison', 'nd','lexington', 'amoeba', 'lenox', 'malcolm', 'broadway', 'amsterdam', 'christoph',"ashburi", "uw", "midtown"]




In [68]:
stop_words2 = stopwords.words('english')

for word in nyc_landmarks:
    stop_words2.append(word)
    
for word in sf_landmarks:
    stop_words2.append(word)

for word in generic_travel:
    stop_words2.append(word)
    
for word in descrip_words:
    stop_words2.append(word)
    
for word in add_words:
    stop_words2.append(word)

In [69]:
from sklearn.feature_extraction.text import  TfidfVectorizer 
# ngram_range=(1,2)
tfidf2 = TfidfVectorizer(tokenizer = custom_tokenizer,stop_words = stop_words2, token_pattern = "\\b[a-zA-Z][a-zA-Z]+\\b", min_df = .01, max_df = .95 )
tfidf_vecs2 = tfidf2.fit_transform(df4["revs"])



/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'doesnt', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldv', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youv'] not in stop_words.
  sorted(inconsistent))


In [70]:
tfidf_3 = tfidf_vecs2.todense()

tfidf_df3 = pd.DataFrame(tfidf_vecs2.todense(), columns = tfidf2.get_feature_names())

tfidf_df3.shape

(27, 17116)

In [71]:
from sklearn.decomposition import NMF
nmf2 = NMF(n_components=27)
nmf_vecs2 = nmf2.fit_transform(tfidf_vecs2)

In [72]:
topic_words3 = []
for r in nmf2.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:10]
    topic_words3.append([tfidf_df3.iloc[:,a[e][1]].name for e in range(len(a))])


In [73]:
for i in topic_words3:
    print(i)

['italian', 'coit', 'church', 'tower', 'bakeri', 'gelato', 'pizza', 'pastri', 'molinari', 'babylon']
['lion', 'chowder', 'seafood', 'clam', 'seal', 'boudin', 'crab', 'touristi', 'boat', 'sourdough']
['river', 'metropolitan', 'frick', 'mansion', 'mile', 'met', 'guggenheim', 'carl', 'schurz', 'neue']
['sak', 'window', 'tiffani', 'patrick', 'trump', 'rockefel', 'brand', 'tower', 'cathedr', 'display']
['steiner', 'mansion', 'victorian', 'fillmor', 'robin', 'alta', 'lyon', 'hilli', 'drive', 'william']
['gay', 'harvey', 'rainbow', 'milk', 'flag', 'lgbt', 'glbt', 'nake', 'theatr', 'communiti']
['hipster', 'l', 'river', 'hip', 'smorgasburg', 'vintag', 'flea', 'breweri', 'young', 'trendi']
['apollo', 'church', 'sylvia', 'gospel', 'soul', 'brownston', 'theatr', 'black', 'rooster', 'theater']
['convent', 'moscon', 'truck', 'basebal', 'metreon', 'dotcom', 'cal', 'club', 'financi', 'dynam']
['fairmont', 'cathedr', 'grace', 'hopkin', 'steep', 'mark', 'california', 'huntington', 'climb', 'powel']
['l

In [74]:
doc_cluster2 = nmf_vecs2.argmax(axis = 1)

print(doc_cluster2.shape)
print(df4.shape)

(27,)
(27, 2)


In [75]:
clusters2 = pd.DataFrame(doc_cluster2)

print(clusters2.shape)


(27, 1)


In [76]:
clusters2.reset_index(inplace=True)

In [77]:
df4.reset_index(inplace=True)

In [78]:
df5 = df4.merge(clusters2, how="outer", on="index")

In [79]:
df5[df5[0]==0]

,index,neighb,revs,0
22,22,north_beach,We had a wonderful dinner at The North Beach R...,0


In [85]:
df5.columns

Index(['index', 'neighb', 'revs', 0], dtype='object')

In [86]:
df5.columns = ["index", "neighb", "revs", "cat"]#, "neighb2"]#, "words"]

In [87]:
df5["words"] = 0

In [228]:
df5.loc[df5["cat"]==0, "words"] = str(topic_words3[0])

In [132]:
import ast

In [130]:
def get_attributes(x):
    x = list(df5[df5["neighb2"]==x]["words"].values)[0]
    x = ast.literal_eval(x)
    return x

In [255]:
get_attributes("Upper West Side")

['Nature',
 'Brownstone',
 'Hudson',
 'Zabar',
 'Columbia',
 'River',
 'Circle',
 'Dakota',
 'Residential',
 'Lennon']

In [88]:
for i in range(0,27):
#     df5.loc[df5["cat"]==i, "words"] = topic_words3[i]
    df5.loc[df5["cat"]==i, "words"] = str(topic_words3[i])

## Combine Review Clusters by Neighborhood


### Categories
1. park
2. store, shopping
3. views
4. bars/ going out
5. beautiful/ picturesque
6. busy, bustling
7. historical
8. cool, trendy 13. vibe, young
9. RECATEGORIZE
10. architecture
11. museum
12. art galleries
13. vibe, young
14. relax

In [137]:
nmf_vecs.shape

(15454, 14)

In [138]:
df2.shape

(15454, 11)

In [139]:
bad_indexes = df2[df2[0]==8].index

In [140]:
bad_indexes

Int64Index([    1,     5,     8,     9,    11,    13,    16,    17,    21,
               23,
            ...
            15429, 15433, 15434, 15436, 15439, 15440, 15446, 15449, 15450,
            15452],
           dtype='int64', length=5876)

In [141]:
nmf_df = pd.DataFrame(nmf_vecs)

In [142]:
nmf_df.reset_index(inplace=True)

In [143]:
df_filters = df2[~(df2["index"].isin(bad_indexes))]

In [144]:
df_filters.shape

(9578, 11)

In [145]:
nmf_filtered = nmf_df[~nmf_df["index"].isin(bad_indexes)]

In [146]:
nmf_filtered.shape

(9578, 15)

In [147]:
nmf_filtered.index

Int64Index([    0,     2,     3,     4,     6,     7,    10,    12,    14,
               15,
            ...
            15438, 15441, 15442, 15443, 15444, 15445, 15447, 15448, 15451,
            15453],
           dtype='int64', length=9578)

In [148]:
nmf_filtered[8]=0

/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [149]:
import numpy as np

In [150]:
nmf_vecs_clean = np.array(nmf_filtered.drop("index", axis=1))

In [151]:
nmf_vecs_clean.shape

(9578, 14)

In [152]:
df_filters.shape

(9578, 11)

In [153]:
my_neighbs = list(df_filters.neighb.unique())

In [154]:
my_neighbs

['chelsea',
 'chinatown',
 'east_village',
 'fidi',
 'fifth_ave',
 'greenwich_village',
 'harlem',
 'soho',
 'times_sq',
 'tribeca',
 'union_sq',
 'ues',
 'uws',
 'west_village',
 'williamsburg',
 'castro',
 'chinatown_sf',
 'embarcadero',
 'fisherman_wharf',
 'haight_ashbury',
 'mission_district',
 'nob_hill',
 'north_becah',
 'pacific_heights',
 'presidio',
 'soma',
 'union_sq_sf']

In [155]:
my_neighbs_dict = {}

In [156]:
df_filters.reset_index(inplace=True)

In [157]:
for i in my_neighbs:
    my_neighbs_dict[i] = [df_filters[df_filters["neighb"]==i].index[0],df_filters[df_filters["neighb"]==i].index[-1]]

In [158]:
my_neighbs_dict

{'castro': [5335, 5621],
 'chelsea': [0, 133],
 'chinatown': [134, 446],
 'chinatown_sf': [5622, 6088],
 'east_village': [447, 550],
 'embarcadero': [6089, 6477],
 'fidi': [551, 986],
 'fifth_ave': [987, 1286],
 'fisherman_wharf': [6478, 6975],
 'greenwich_village': [1287, 2037],
 'haight_ashbury': [6976, 7602],
 'harlem': [2038, 2211],
 'mission_district': [7603, 7838],
 'nob_hill': [7839, 7941],
 'north_becah': [7942, 8050],
 'pacific_heights': [8051, 8106],
 'presidio': [8107, 8985],
 'soho': [2212, 2896],
 'soma': [8986, 9006],
 'times_sq': [2897, 3350],
 'tribeca': [3351, 3431],
 'ues': [3874, 3963],
 'union_sq': [3432, 3873],
 'union_sq_sf': [9007, 9577],
 'uws': [3964, 4274],
 'west_village': [4275, 4822],
 'williamsburg': [4823, 5334]}

In [159]:
my_neighbs_dict.keys()

dict_keys(['chelsea', 'chinatown', 'east_village', 'fidi', 'fifth_ave', 'greenwich_village', 'harlem', 'soho', 'times_sq', 'tribeca', 'union_sq', 'ues', 'uws', 'west_village', 'williamsburg', 'castro', 'chinatown_sf', 'embarcadero', 'fisherman_wharf', 'haight_ashbury', 'mission_district', 'nob_hill', 'north_becah', 'pacific_heights', 'presidio', 'soma', 'union_sq_sf'])

In [160]:
my_neighb_avg = {}

In [161]:
my_neighbs_dict["chelsea"][1]

133

In [162]:
my_neighbs_dict["chelsea"]

[0, 133]

In [163]:
np.mean(nmf_vecs_clean[my_neighbs_dict["chelsea"][0]:my_neighbs_dict["chelsea"][1]], axis=0)

array([0.00984984, 0.01125834, 0.0040928 , 0.02208379, 0.00932467,
       0.0048427 , 0.00403933, 0.01067217, 0.        , 0.00705729,
       0.00692212, 0.04758333, 0.00912378, 0.00457576])

In [164]:
for x in my_neighbs:
    my_neighb_avg[x] = np.mean(nmf_vecs_clean[(my_neighbs_dict[x][0]):(my_neighbs_dict[x][1]+1)],axis=0)

In [165]:
rec_df = pd.DataFrame(my_neighb_avg).transpose()

In [166]:
rec_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
castro,0.002475,0.017854,0.003422,0.042867,0.007340,0.007303,0.032051,0.009384,0.0,0.004652,0.010623,0.003441,0.012923,0.009978
chelsea,0.009784,0.011174,0.004087,0.021919,0.009285,0.004814,0.004028,0.010593,0.0,0.007005,0.006999,0.047239,0.009059,0.004559
chinatown,0.009005,0.027895,0.001599,0.001508,0.004317,0.025329,0.004274,0.008867,0.0,0.003058,0.004668,0.002940,0.003480,0.005393
chinatown_sf,0.005209,0.035232,0.002748,0.003972,0.008439,0.018004,0.009047,0.014086,0.0,0.013815,0.003834,0.007981,0.002237,0.005072
east_village,0.009508,0.015481,0.002404,0.046174,0.006868,0.006077,0.008556,0.021171,0.0,0.005067,0.016490,0.015945,0.009301,0.007222
embarcadero,0.009440,0.007821,0.047745,0.008752,0.018391,0.011522,0.002811,0.005583,0.0,0.002667,0.007655,0.005514,0.001820,0.008424
fidi,0.022623,0.003208,0.026636,0.008132,0.012787,0.013333,0.011637,0.005711,0.0,0.007309,0.023016,0.002344,0.001170,0.004435
fifth_ave,0.013542,0.051474,0.003350,0.002229,0.016041,0.013759,0.001766,0.003308,0.0,0.008673,0.010920,0.004613,0.002337,0.001349
fisherman_wharf,0.006369,0.009872,0.030416,0.010704,0.006868,0.019607,0.003533,0.005997,0.0,0.000585,0.015661,0.002566,0.003179,0.007576
greenwich_village,0.022281,0.006999,0.002703,0.024637,0.017807,0.006769,0.011533,0.011507,0.0,0.011685,0.003564,0.005384,0.009354,0.017737


In [167]:
nmf_vecs_clean.shape

(9578, 14)

In [168]:
rec_df.shape

(27, 14)

In [169]:
df_filters.shape

(9578, 12)

In [170]:
sf = ['castro','chinatown_sf',
       'embarcadero','fisherman_wharf',
       'haight_ashbury', 'mission_district',
       'nob_hill', 'north_becah', 'pacific_heights', 'presidio',
       'soma','union_sq_sf']



In [171]:
df2[df2["city"]=="SF"].neighb.nunique()

12

In [191]:
['castro', 'chelsea', 'chinatown', 'chinatown_sf', 'east_village',
       'embarcadero', 'fidi', 'fifth_ave', 'fisherman_wharf',
       'greenwich_village', 'haight_ashbury', 'harlem', 'mission_district',
       'nob_hill', 'north_becah', 'pacific_heights', 'presidio', 'soho',
       'soma', 'times_sq', 'tribeca', 'ues', 'union_sq', 'union_sq_sf', 'uws',
       'west_village', 'williamsburg']

['castro',
 'chelsea',
 'chinatown',
 'chinatown_sf',
 'east_village',
 'embarcadero',
 'fidi',
 'fifth_ave',
 'fisherman_wharf',
 'greenwich_village',
 'haight_ashbury',
 'harlem',
 'mission_district',
 'nob_hill',
 'north_becah',
 'pacific_heights',
 'presidio',
 'soho',
 'soma',
 'times_sq',
 'tribeca',
 'ues',
 'union_sq',
 'union_sq_sf',
 'uws',
 'west_village',
 'williamsburg']

In [172]:
rec_df.loc[(rec_df.index.isin(sf)), "city"]= "sf"
rec_df.loc[~(rec_df.index.isin(sf)), "city"]= "ny"

In [173]:
rec_ny = rec_df[rec_df["city"]=="ny"]
rec_sf = rec_df[rec_df["city"]=="sf"]

In [174]:
rec_df.shape

(27, 15)

In [175]:
rec_ny.shape

(15, 15)

In [176]:
rec_sf.shape

(12, 15)

In [177]:
rec_ny_arr = np.array(rec_ny[[0,1,2,3,4,5,6,7,8,9,10,11,12,13]])

In [178]:
rec_sf_arr = np.array(rec_sf[[0,1,2,3,4,5,6,7,8,9,10,11,12,13]])

In [179]:
rec_sf_arr.shape

(12, 14)

In [180]:
from sklearn.metrics.pairwise import linear_kernel

#Compute the cosine similarity matrix of nmf vectors
cosine_sim2 = linear_kernel(rec_ny_arr, rec_sf_arr)

In [181]:
cosine_sim2.shape

(15, 12)

In [182]:
cosine_sim_df = pd.DataFrame(cosine_sim2)

In [183]:
cosine_sim_df.columns = ['castro', 'chinatown_sf', 'embarcadero', 'fisherman_wharf',
       'haight_ashbury', 'mission_district', 'nob_hill', 'north_becah',
       'pacific_heights', 'presidio', 'soma', 'union_sq_sf']

In [184]:
cosine_sim_df.index = ['chelsea', 'chinatown', 'east_village', 'fidi', 'fifth_ave',
       'greenwich_village', 'harlem', 'soho', 'times_sq', 'tribeca', 'ues',
       'union_sq', 'uws', 'west_village', 'williamsburg']

In [185]:
rec_sf.index ##fisherman wharf

Index(['castro', 'chinatown_sf', 'embarcadero', 'fisherman_wharf',
       'haight_ashbury', 'mission_district', 'nob_hill', 'north_becah',
       'pacific_heights', 'presidio', 'soma', 'union_sq_sf'],
      dtype='object')

In [186]:
def get_rec(name, city):
    if city=="ny":
        return cosine_sim_df.loc[name,:].sort_values(ascending=False)[:3].index[0], cosine_sim_df.loc[name,:].sort_values(ascending=False)[:3].index[1],cosine_sim_df.loc[name,:].sort_values(ascending=False)[:3].index[2]
    else:
        return cosine_sim_df.loc[:,name].sort_values(ascending=False)[:3].index[0], cosine_sim_df.loc[:,name].sort_values(ascending=False)[:3].index[1],cosine_sim_df.loc[:,name].sort_values(ascending=False)[:3].index[2]


In [187]:
print("castro",get_rec("castro", "sf"))
print("chinatown_sf",get_rec("chinatown_sf", "sf"))
print("embarcadero",get_rec("embarcadero", "sf"))
print("fisherman_wharf",get_rec("fisherman_wharf", "sf"))
print("haight_ashbury",get_rec("haight_ashbury", "sf"))
print("mission_district",get_rec("mission_district", "sf"))
print("nob_hill",get_rec("nob_hill", "sf"))
print("north_becah",get_rec("north_becah", "sf"))
print("pacific_heights",get_rec("pacific_heights", "sf"))
print("presidio",get_rec("presidio", "sf"))
print("soma",get_rec("soma", "sf"))
print("union_sq_sf",get_rec("union_sq_sf", "sf"))


castro ('east_village', 'williamsburg', 'west_village')
chinatown_sf ('fifth_ave', 'soho', 'times_sq')
embarcadero ('fidi', 'ues', 'williamsburg')
fisherman_wharf ('ues', 'fidi', 'times_sq')
haight_ashbury ('soho', 'fifth_ave', 'east_village')
mission_district ('chelsea', 'soho', 'east_village')
nob_hill ('fidi', 'ues', 'uws')
north_becah ('east_village', 'williamsburg', 'uws')
pacific_heights ('fidi', 'ues', 'uws')
presidio ('ues', 'fidi', 'uws')
soma ('ues', 'uws', 'east_village')
union_sq_sf ('fifth_ave', 'soho', 'ues')


In [188]:
print("chelsea", get_rec("chelsea", "ny"))
print("chinatown", get_rec("chinatown", "ny"))
print("east_village", get_rec("east_village", "ny"))
print("fidi", get_rec("fidi", "ny"))
print("fifth_ave", get_rec("fifth_ave", "ny"))
print("greenwich_village", get_rec("greenwich_village", "ny"))
print("harlem", get_rec("harlem", "ny"))
print("soho", get_rec("soho", "ny"))
print("times_sq", get_rec("times_sq", "ny"))
print("tribeca", get_rec("tribeca", "ny"))
print("ues", get_rec("ues", "ny"))
print("union_sq", get_rec("union_sq", "ny"))
print("uws", get_rec("uws", "ny"))
print("west_village", get_rec("west_village", "ny"))
print("williamsburg", get_rec("williamsburg", "ny"))

chelsea ('mission_district', 'soma', 'castro')
chinatown ('union_sq_sf', 'chinatown_sf', 'haight_ashbury')
east_village ('soma', 'castro', 'mission_district')
fidi ('presidio', 'pacific_heights', 'nob_hill')
fifth_ave ('union_sq_sf', 'chinatown_sf', 'haight_ashbury')
greenwich_village ('castro', 'pacific_heights', 'soma')
harlem ('presidio', 'castro', 'pacific_heights')
soho ('mission_district', 'union_sq_sf', 'haight_ashbury')
times_sq ('chinatown_sf', 'fisherman_wharf', 'union_sq_sf')
tribeca ('pacific_heights', 'soma', 'mission_district')
ues ('soma', 'presidio', 'pacific_heights')
union_sq ('soma', 'union_sq_sf', 'presidio')
uws ('soma', 'presidio', 'pacific_heights')
west_village ('pacific_heights', 'castro', 'soma')
williamsburg ('castro', 'soma', 'mission_district')


## Photos

In [189]:
from vgg16_places_365 import VGG16_Places365
model = VGG16_Places365(weights='places', include_top=False)

/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [190]:
image_sf = {}
image_ny = {}

In [191]:
from PIL import Image
import os
# import urllib2
import numpy as np
from PIL import Image
from cv2 import resize
from scipy.spatial import distance

In [195]:
def get_images(neighb, num):
    for i in range(1,num):
        image = Image.open("Photos/NYC/"+neighb+"/"+str(i)+neighb+".jpg")
        image = np.array(image, dtype=np.uint8)
        image = resize(image, (224, 224))
        image = np.expand_dims(image, 0)
        a = model.predict(image)
        image_ny[neighb+str(i)] = np.reshape(a,-1)

In [193]:
get_images("castro", 18)
get_images("chinatown_sf",14)
get_images("embarcadero",11)
get_images("fisherman", 11)
get_images("haight_ashbury", 11)
get_images("mission", 30)
get_images("nobhill", 13)
get_images("north_beach", 11)
get_images("pacific_heights",14)
get_images("presidio", 10)
get_images("soma", 22)

In [196]:
get_images("chelsea", 11)
get_images("chinatown", 9)
get_images("eastvill", 19)
get_images("fidi", 20)
get_images("fifth_ave", 11)
get_images("greenvill", 13)
get_images("harl", 11)
get_images("soho", 13)
get_images("timessq", 13)
get_images("tribeca", 16)
get_images("ues", 12)
get_images("unionsq", 13)
get_images("uws", 20)
get_images("west_vill", 12)
get_images("will", 19)

In [197]:
image_ny_df = pd.DataFrame(image_ny).transpose()
image_sf_df = pd.DataFrame(image_sf).transpose()

In [198]:
def get_rec_photo(name):
    a = image_ny_df.loc[name].values
    dist = []
    for i in range(0,74):
        b = image_sf_filtered.iloc[i].values
        dist.append((i, distance.euclidean(a,b)))
    dist.sort(key=lambda tup:tup[1])
    my_rec_loc = []
    c = dist[:3]
    for i in c:
        my_rec_loc.append(i[0])
    imgs = image_sf_filtered.iloc[my_rec_loc]
    d = imgs.iloc[0].name
    try:
        num = int(d[-2:])
        name = (str(num)+d[:-2]+".jpg")
    except:
        num = int(d[-1:])
        name = (str(num)+d[:-1]+".jpg")
    return name

In [418]:
image_ny_filtered.shape

(95, 25088)

In [203]:
def get_rec_photo2(name):
    a = image_sf_filtered.loc[name].values
    dist = []
    for i in range(0,90):
        b = image_ny_filtered.iloc[i].values
        dist.append((i, distance.euclidean(a,b)))
    dist.sort(key=lambda tup:tup[1])
    my_rec_loc = []
    c = dist[:3]
    for i in c:
        my_rec_loc.append(i[0])
    imgs = image_ny_filtered.iloc[my_rec_loc]
    d = imgs.iloc[0].name
    d1 = imgs.iloc[1].name
    try:
        num = int(d[-2:])
        name = (str(num)+d[:-2]+".jpg")
    except:
        num = int(d[-1:])
        name = (str(num)+d[:-1]+".jpg")
    return name

In [199]:
good_images_sf = ["castro1", "castro3", "castro4", "castro5", "castro6", "castro11", "castro14","castro15","castro16", "castro17", "chinatown_sf1", "chinatown_sf2", "chinatown_sf3", "chinatown_sf6", "chinatown_sf7","chinatown_sf8","chinatown_sf9", "chinatown_sf10","chinatown_sf11","chinatown_sf13",\
                 "embarcadero1", "embarcadero2","embarcadero5","embarcadero7","embarcadero9","fisherman1","fisherman6","fisherman7","fisherman8","fisherman10",\
                 "haight_ashbury3","haight_ashbury7","haight_ashbury9","mission4","mission5","mission6","mission7","mission12","mission13","mission14","mission21","mission26","mission29","nobhill1","nobhill4","nobhill5","nobhill7",\
"nobhill10","north_beach1",  "north_beach10","pacific_heights1","pacific_heights2","pacific_heights5","pacific_heights6","pacific_heights7","pacific_heights9","pacific_heights10",\
                 "pacific_heights12", "pacific_heights13","presidio1","presidio3","presidio6","presidio7","presidio9","presidio10","soma1","soma2",\
                 "soma4","soma7","soma9","soma10","soma12","soma13","soma14","soma17"]


In [200]:
good_images_ny = ['chelsea1','chelsea10','chelsea2','chelsea3','chelsea5','chelsea6','chelsea7','chelsea8', \ 
 'chinatown1', 'chinatown2','chinatown3','chinatown7','chinatown8','eastvill1','eastvill10','eastvill13','eastvill5','eastvill6',  'eastvill9', \
 'fidi12','fidi18','fidi19','fidi2','fidi6','fidi9','fifth_ave3','fifth_ave5','fifth_ave6','fifth_ave8','greenvill1','greenvill10', \                  
 'greenvill12','greenvill2','greenvill4', 'greenvill6','greenvill7', 'greenvill9','harl10','harl8','soho1','soho10', \ 
'soho12','soho4','timessq1','timessq11','timessq6','timessq9','tribeca1','tribeca2','tribeca5', \ 
 'tribeca6','tribeca9','ues10','ues2','ues3','ues4','ues6','ues7','ues8','unionsq10','unionsq2','unionsq6','unionsq8','unionsq9', \ 
 'uws11', 'uws13', 'uws14', 'uws15', 'uws16', 'uws17', 'uws19', 'uws2', 'uws3', 'uws7', 'west_vill1','west_vill11', \ 
 'west_vill2', 'west_vill5', 'west_vill9','will1', 'will10', 'will11', 'will12', 'will13','will14','will15','will18','will5','will7','will9']

In [201]:
image_sf_filtered = image_sf_df[image_sf_df.index.isin(good_images_sf)]

In [202]:
image_ny_filtered = image_ny_df[image_ny_df.index.isin(good_images_ny)]

In [150]:
get_rec_photo("chelsea10")

'13pacific_heights.jpg'

In [205]:
def name_trans(x):
    try:
        num = int(x[:2])
        name = (x[2:-4]+str(num))
    except:
        num = int(x[0])
        name = (x[1:-4]+str(num))
    return name

## Build a Flask App

In [234]:
all_neighbs_ny = list(df3[df3["city"]=="nyc"].neighb.unique())

In [240]:
all_neighbs_sf = list(df3[df3["city"]=="SF"].neighb.unique())

In [ ]:
def get_folder(a):
    try:
        x = int(a[:2])
        y = a[2:].split(".")[0]
    except:
        y = a[1:].split(".")[0]
#     if y =="will":
#         return "Williamsburg"
    return y.capitalize()

In [257]:
from flask import Flask, request, render_template, flash, redirect, url_for, jsonify

from flask_wtf import Form
from wtforms import TextField, IntegerField, TextAreaField, SubmitField, RadioField,SelectField
from wtforms.validators import DataRequired
from flask_sqlalchemy import SQLAlchemy  
from werkzeug import secure_filename
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////tmp/test.db'
app.config['SECRET_KEY'] = 'you-will-never-guess'


UPLOAD_FOLDER = '/Users/elizabethoriordan/Desktop/data_science/metis/Proj_5/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

db = SQLAlchemy(app)
class Neighbs(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    city = db.Column(db.String(64), index=True)
    neighb = db.Column(db.String(64), index=True)
    
    def __repr__(self):
        return '<Neighbs {}>'.format(self.neighb)
    
# db.create_all()
# for i in all_neighbs_ny:
#     n = Neighbs(city="nyc", neighb=i)
#     db.session.add(n)
#     db.session.commit()
    
# for i in all_neighbs_sf:
#     n = Neighbs(city="sf", neighb=i)
#     db.session.add(n)
#     db.session.commit()

class ContactForm(Form):
    city = SelectField('Cities',choices=[('nyc', 'NYC'), ('sf', 'SF')], validators = [DataRequired()])
    neighb = SelectField('Neighborhood', choices = [],validators = [DataRequired()])
    submit = SubmitField("Submit")

class AttrForm(Form):
    city = SelectField('Cities',choices=[('nyc', 'NYC'), ('SF', 'SF')], validators = [DataRequired()])
    att1 = SelectField('Attribute 1:',choices=[('park', 'Park'), ('shopping', 'Shopping'), ('views', 'Views'), ('bars', 'Bars'),('busy/ bustling', 'Busy'),('historical', 'Historical'),('cool', 'Cool'),('art', 'Art'),('museums', 'Museums'),('young vibe', 'Young Vibe'),('relaxing', 'Relaxing')], validators = [DataRequired()])
    att2 = SelectField('Attribute 2:',choices=[('park', 'Park'), ('shopping', 'Shopping'), ('views', 'Views'), ('bars', 'Bars'),('busy/ bustling', 'Busy'),('historical', 'Historical'),('cool', 'Cool'),('art', 'Art'),('museums', 'Museums'),('young vibe', 'Young Vibe'),('relaxing', 'Relaxing')], validators = [DataRequired()])
    submit = SubmitField("Submit")

@app.route('/')
@app.route('/index')
def index():
    
    return render_template('index.html')

@app.route('/rec',methods=['GET', 'POST'])
def rec():
    form = ContactForm()
    form.neighb.choices = [(x.id, x.neighb) for x in Neighbs.query.filter_by(city="nyc").all()]
    if request.method == 'POST':
        my_id =  form.neighb.data
        my_neighb =  Neighbs.query.filter_by(id=my_id).first().neighb
        return redirect(url_for('results', neighbor=my_neighb))#,my_neighb = form.neighb.data))
    return render_template('rec.html', form=form)

@app.route('/results')
@app.route('/results/<neighbor>')
def results(neighbor):
    my_city = Neighbs.query.filter_by(neighb=neighbor).first().city
    my_city = my_city[0:2].lower()
    my_result = get_rec(neighbor,my_city)
    my_result = list(my_result)
    for i in range(0,3):
        my_result[i] = my_result[i].split("_")
        my_result[i] = [j.capitalize() for j in my_result[i]]
        my_result[i] = " ".join(my_result[i])
    return render_template('results.html', result = my_result)

@app.route('/neighbo/<city>')
def neighbo(city):
    neighbo = Neighbs.query.filter_by(city=city).all()
    
    neighbArray = []
    for neighb in neighbo:
        neighbObj = {}
        neighbObj['id'] = neighb.id
        neighbObj['name'] = neighb.neighb
        neighbArray.append(neighbObj)
    return jsonify({'neighbs': neighbArray})

@app.route('/map',methods=['GET', 'POST'])
def map():
    return render_template('map.html')

@app.route('/score',methods=['GET', 'POST'])
def score():
    if request.method == 'POST':
        data = request.json
        x = data["example"]
        print(x[0])
        results = get_attributes(x[0])
        print(results)
        a = get_attributes("Upper West Side")
        return jsonify(results)
        return "a"
#         return render_template('map.html', res=result)
#     return render_template('map.html', res=res)

@app.route('/tableau',methods=['GET', 'POST'])
def tableau():
    return render_template('tableau.html')


@app.route('/upload_file', methods=['GET','POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        f.save(os.path.join(app.config['UPLOAD_FOLDER'], f.filename))
        name = f.filename
        name = name_trans(name)
        my_rec =  get_rec_photo2(name)
        return redirect(url_for('me', apath = my_rec))
    return render_template('upload_file.html')

@app.route('/rec_att',methods=['GET', 'POST'])
def rec_att():
    form = AttrForm()
    if request.method == 'POST':
        my_city = form.city.data
        my_att1 =  form.att1.data
        my_att2 =  form.att2.data
        recs = get_rec_by_attr(my_att1, my_att2, my_city)
        mapping = {'chelsea': "Chelsea", 'chinatown': "Chinatown", 'east_village': "East Village", 'fidi': "Financial District", 'fifth_ave': "Fifth Avenue", 
       'greenwich_village': "Greenwich Village", 'harlem': "Harlem", 'soho': "SoHo", 'times_sq': "Times Square", 'tribeca': "Tribeca", 
       'union_sq': "Union Square", 'ues': "Upper East Side", 'uws': "Upper West Side", 'west_village': "West Village", 'williamsburg': "Williamsburg", 'castro': "Castro", 
       'chinatown_sf': "Chinatown SF", 'embarcadero': "Embarcadero/ Fidi", 'fisherman_wharf': "Fisherman's Wharf", 'haight_ashbury': "Haight-Ashbury", 
       'mission_district': "Mission District", 'nob_hill': "Nob Hill", 'north_beach': "North Beach", 'pacific_heights': "Pacific Heights", 
       'presidio': "Presidio", 'soma': "Soma", 'union_sq_sf': "Union Square SF"}
        recs = list(recs)
        recs[0] = mapping[recs[0]]
        recs[1] = mapping[recs[1]]
        return render_template('results2.html', result=recs)

    return render_template('rec_att.html', form=form)

@app.route('/results2')
@app.route('/results2/<rec>')
def results2(rec):
    a = rec[0]
    b = rec[1]
    return render_template('results2.html', result = rec)

    
@app.route('/me/<apath>')
def me(apath):
    fold = get_folder(apath)
    x = '/static/NYC/'+fold + '/'+ apath
    if fold=="'will'":
        fold="Williamsburg"
    return render_template('me.html', user_image = x, user_tag = "Williamsburg")



if __name__ == '__main__':
    app.run()


/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:158: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
127.0.0.1 - - [12/Dec/2018 20:00:45] "GET /rec_att HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2018 20:00:59] "GET /index HTTP/1.1" 200 -
/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: FlaskWTFDeprecationWarning: "flask_wtf.Form" has been renamed to "FlaskForm" and will be removed in 1.0.
127.0.0.1 - - [12/Dec/2018 20:01:50] "GET /rec HTTP/1.1" 200 -
127.0.0.1 - - [

127.0.0.1 - - [12/Dec/2018 20:04:14] "POST /score HTTP/1.1" 500 -


Rosedale
Springfield Gardens
South Ozone Park
Richmond Hill


[2018-12-12 20:04:14,271] ERROR in app: Exception on /score [POST]
Traceback (most recent call last):
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functi

Forest Park
Glendale
Middle Village
Ridgewood
Williamsburg
['Hipster', 'L', 'River', 'Hip', 'Smorgasburg', 'Vintage', 'Flea', 'Brewery', 'Young', 'Trendy']


...

[2018-12-12 20:17:36,234] ERROR in app: Exception on /score [POST]
Traceback (most recent call last):
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functi

[2018-12-12 20:17:36,351] ERROR in app: Exception on /score [POST]
Traceback (most recent call last):
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functi

Long Island City
Astoria
Jackson Heights
Corona
Kew Gardens Hills
Jamaica Estates
Cambria Heights


[2018-12-12 21:01:00,911] ERROR in app: Exception on /score [POST]
Traceback (most recent call last):
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/elizabethoriordan/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functi

Broad Channel
